In [ ]:
import pandas as pd
student_data=pd.read_csv("/content/student_resolution.csv",)

student_data.duplicated().sum()

np.int64(0)

In [ ]:
student_data.columns

Index(['Genre', 'Reports', 'Age', 'GPA', 'Year', 'Count', 'Gender',
       'Nationality', 'Resolution'],
      dtype='object')

In [ ]:
student_data["Nationality"].unique()

array(['France', 'Mexico', 'South Africa', 'Yemen', 'Indonesia', 'Egypt',
       'Soudan'], dtype=object)

generate student name and ticket_id

In [ ]:
pip install Faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 26.0 MB/s eta 0:00:00


In [ ]:
student_data.drop(columns=["GPA"],axis=1,inplace=True)

student_data["Gender"].apply(lambda x:"Male" if x=="M" else "Female")


def convert_yearto_level(year):
  if year==1:
    return "First"
  elif year==2:
    return "Second"
  elif year==3:
    return "Third"
  elif year==4:
    return "Fourth"
  elif year==5:
    return "Fifth"

student_data["Year"].apply(lambda x:convert_yearto_level(x))

,Year
0,Fifth
1,First
2,Fourth
3,Fifth
4,Second
...,...
949,Third
950,Second
951,Second
952,Third


In [ ]:
import pandas as pd
from faker import Faker
import random

fake = Faker()

# Use only locales supported by faker
# generate a random names accourding to region
nationalities = {
    'France': 'en_GB',
    'Mexico': 'es_MX',
    'South Africa': 'en_US',
    'Yemen': 'ar_EG',
    'Indonesia': 'id_ID',
    'Egypt': 'ar_EG',
    'Soudan': 'ar_EG',
    'United States': 'en_US',
    'United Kingdom': 'en_GB',
    'China': 'en_GB',
    'India': 'en_GB',
    'Japan': 'en_GB',
    'Germany': 'de_DE',
    'Brazil': 'pt_BR',
    'Russia': 'ru_RU',
    'Nigeria': 'en_US',
    'Australia': 'en_AU',
    'Canada': 'en_CA',
}

data = {
    'Nationality': [random.choice(list(nationalities.keys())) for _ in range(955)],
}

df = pd.DataFrame(student_data)  # Fix: Use 'data' instead of 'student_data'

# Randomly assign gender to each student
df['Gender'] = [random.choice(['Male', 'Female']) for _ in range(len(df))]

# Function to generate culturally appropriate names based on nationality and gender
def generate_name(nationality, gender):
    locale = nationalities[nationality]
    fake_local = Faker(locale)
    if gender == 'Male':
        return fake_local.first_name_male() + ' ' + fake_local.last_name()
    else:
        return fake_local.first_name_female() + ' ' + fake_local.last_name()

# Generate student names based on nationality and gender
df['Student Name'] = df.apply(lambda row: generate_name(row['Nationality'], row['Gender']), axis=1)

# Generate unique 7-digit ticket IDs

ticket_ids = set()

while len(ticket_ids) < len(df):
    ticket_ids.add(f"TKT-{fake.random_number(digits=6)}")

df['Ticket ID'] = list(ticket_ids)  # Conversion maintains order

# Display the DataFrame
print(df.head())



                               Genre  \
0  International student experiences   
1  International student experiences   
2  International student experiences   
3  International student experiences   
4  International student experiences   

                                             Reports  Age  Year  Count  \
0  4. I appreciate the efforts the university has...   24     5      1   
1  5. I've been trying to improve my language ski...   26     1      1   
2  6. I think it would be helpful if the universi...   23     4      1   
3  7. I appreciate the efforts of my fellow stude...   20     5      1   
4  8. I wish the university offered more language...   27     2      1   

   Gender Nationality                                         Resolution  \
0  Female      France  Create a feedback system for international stu...   
1    Male      France  Provide access to online language courses and ...   
2    Male      France  Organize study groups for international studen...   
3    Male 

In [ ]:
int(df["Ticket ID"].duplicated().sum())

0

In [ ]:
import pandas as pd
import numpy as np
import random
from collections import defaultdict

# Set random seed for reproducibility
np.random.seed(42)
random.seed(42)

# Enhanced GPA generator
def generate_realistic_gpa(year, student_id, all_gpas):
    """
    Generates realistic GPA that:
    - Progresses with academic level
    - Ensures uniqueness within tolerance
    - Adds diversity and avoids close duplicates
    """
    # Multiple GPA clusters for diversity
    year_params = {
        "First": [(2.3, 0.6), (2.6, 0.5)],
        "Second": [(2.6, 0.5), (2.9, 0.4)],
        "Third": [(2.9, 0.4), (3.1, 0.3)],
        "Fourth": [(3.1, 0.3), (3.3, 0.2)],
        "Fifth": [(3.2, 0.25), (3.4, 0.2)],
    }

    max_attempts = 10
    for _ in range(max_attempts):
        mean, std_dev = random.choice(year_params[year])
        gpa = np.random.normal(mean, std_dev)
        gpa += np.random.uniform(-0.02, 0.02)  # Small jitter for diversity
        gpa = np.clip(gpa, 1.5, 3.999)

        if not any(abs(gpa - existing) < 0.03 for existing in all_gpas.values()):
            all_gpas[student_id] = gpa
            return round(gpa, 4)

    # Fallback
    return round(np.clip(np.random.normal(mean, std_dev), 1.5, 3.999), 4)

# Generate student data
global_gpas = {}
data = {
    'Student_ID': [f"STU{954 + i}" for i in range(954)],
    'Year': np.random.choice(
        ["First", "Second", "Third", "Fourth", "Fifth"],
        954,
        p=[0.25, 0.25, 0.2, 0.2, 0.1]
    )
}


for col in data:
    df[col] = data[col]

# Generate GPAs
df['GPA'] = df.apply(
    lambda row: generate_realistic_gpa(
        year=row['Year'],
        student_id=row['Student_ID'],
        all_gpas=global_gpas
    ), axis=1
)

# Validation function
def has_close_duplicates(gpa_series, tolerance=0.05):
    sorted_gpas = sorted(gpa_series)
    for i in range(1, len(sorted_gpas)):
        if abs(sorted_gpas[i] - sorted_gpas[i-1]) < tolerance:
            return True
    return False

# Summary
summary = {
    "GPA Distribution": df.groupby('Year')['GPA'].describe(),
    "Total students": len(df),
    "Unique GPA values": df['GPA'].nunique(),
    "Duplicates within ±0.05": has_close_duplicates(df['GPA']),
    "Minimum GPA": df['GPA'].min(),
    "Maximum GPA": df['GPA'].max()
}

df.head(),
summary


{'GPA Distribution':         count      mean       std     min       25%     50%     75%     max
 Year                                                                       
 Fifth    97.0  3.316085  0.262034  2.5065  3.171200  3.3301  3.4639  3.9578
 First   257.0  2.477312  0.524025  1.5000  2.044000  2.4867  2.8557  3.8449
 Fourth  181.0  3.183961  0.287873  2.3227  3.002000  3.2085  3.3940  3.7678
 Second  219.0  2.730800  0.529970  1.5000  2.352050  2.8292  3.1404  3.9990
 Third   200.0  3.068284  0.355597  2.0740  2.843525  3.1001  3.2944  3.9990,
 'Total students': 954,
 'Unique GPA values': 918,
 'Duplicates within ±0.05': True,
 'Minimum GPA': 1.5,
 'Maximum GPA': 3.999}

In [ ]:
df["GPA"].unique()

array([1.6803, 2.862 , 3.3868, 2.6138, 1.9246, 1.5   , 1.7171, 3.144 ,
       2.6696, 3.429 , 1.8611, 3.1988, 3.6286, 2.9659, 1.9703, 2.9053,
       2.2556, 3.2895, 2.8292, 2.0764, 3.3401, 2.7227, 3.2568, 2.4533,
       2.1453, 3.5821, 2.423 , 3.4688, 2.7788, 2.1968, 3.085 , 2.4917,
       2.3008, 3.0509, 3.5281, 2.9447, 3.0036, 2.0248, 2.553 , 2.9702,
       2.3309, 2.8603, 1.5546, 3.4646, 1.6381, 2.3645, 2.4994, 3.2801,
       3.7312, 3.7998, 2.7533, 3.3301, 3.1418, 3.6613, 3.8954, 1.9862,
       1.8186, 2.107 , 2.4737, 1.933 , 3.1989, 2.6891, 3.1155, 1.5965,
       3.0051, 1.7497, 2.7436, 2.6178, 3.4605, 2.3227, 2.9854, 2.6923,
       3.1833, 3.2446, 3.3061, 3.2184, 2.5032, 1.7829, 3.4543, 3.07  ,
       2.9349, 2.4059, 2.3448, 1.8926, 3.1073, 3.2052, 3.3807, 2.9337,
       2.9694, 3.4798, 3.4022, 2.8263, 3.164 , 2.5838, 2.8503, 2.7808,
       2.6168, 3.2978, 2.6517, 1.8377, 3.4786, 3.9578, 2.2265, 2.5185,
       3.174 , 2.5858, 2.0071, 3.1463, 3.2614, 3.8032, 3.0297, 3.4845,
      

In [ ]:
df.iloc[45]

,45
Genre,International student experiences
Reports,It can be tough to adapt to a new culture and ...
Age,21
Year,Third
Count,1
Gender,Male
Nationality,Mexico
Resolution,Provide cultural orientation programs to help ...
Student Name,Eugenio Echeverría
Ticket ID,TKT-553052


In [ ]:
df.to_csv('full_student_complaints.csv', index=False)

In [ ]:
df.columns

Index(['Genre', 'Reports', 'Age', 'Year', 'Count', 'Gender', 'Nationality',
       'Resolution', 'Student Name', 'Ticket ID', 'Student_ID', 'GPA'],
      dtype='object')

In [ ]:
df["Genre"].unique()

array(['International student experiences', 'Online learning',
       'Student Affairs', 'Housing and Transportation',
       'Activities and Travelling', 'Food and Cantines',
       'Academic Support and Resources', 'Athletics and sports',
       'Career opportunities', 'Financial Support',
       'Health and Well-being Support'], dtype=object)

data augmentation

In [ ]:
pip install nlpaug

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 10.9 MB/s eta 0:00:00


In [ ]:
pip install nltk

In [ ]:
!pip install textattack
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 52.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 47.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 445.7/445.7 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
df.columns

Index(['Genre', 'Reports', 'Age', 'Year', 'Count', 'Gender', 'Nationality',
       'Resolution', 'Student Name', 'Ticket ID', 'Student_ID', 'GPA'],
      dtype='object')

In [ ]:
!pip install textattack

In [ ]:
!pip install country-converter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 3.3 MB/s eta 0:00:00


In [ ]:
pip install textattack

In [ ]:
pip install nltk

In [ ]:
pip install --upgrade textattack flair

In [ ]:
import pandas as pd
import numpy as np
import random
from faker import Faker
from textattack.augmentation import WordNetAugmenter
from tqdm import tqdm
import hashlib
from faker.providers import person

fake = Faker()
text_augmenter = WordNetAugmenter()

# Add nationality-specific name providers
fake.add_provider(person)

# Cache for tracking uniqueness
response_cache = set()
resolution_cache = set()

# Enhanced nationality to name mapping with proper locales
NATIONALITY_NAME_MAPPING = {
    'United States': 'en_US',
    'United Kingdom': 'en_GB',
    'China': 'en_US',
    'India': 'en_US',
    'Japan': 'en_US',
    'Germany': 'de_DE',
    'France': 'en_US',
    'Brazil': 'pt_BR',
    'Mexico': 'es_MX',
    'Russia': 'ru_RU',
    'Egypt': 'ar_EG',
    'Nigeria': 'en_NG',
    'South Africa': 'en_US',
    'Australia': 'en_AU',
    'Canada': 'en_CA',
    'South Korea': 'en_US',
    'Italy': 'it_IT',
    'Spain': 'es_ES',
    'Saudi Arabia': 'ar_SA',
    'Turkey': 'en_US'
}

# Country weights for more realistic distribution
COUNTRY_WEIGHTS = {
    'United States': 0.25,
    'United Kingdom': 0.1,
    'China': 0.15,
    'India': 0.15,
    'Japan': 0.05,
    'Germany': 0.05,
    'France': 0.05,
    'Brazil': 0.03,
    'Mexico': 0.03,
    'Russia': 0.02,
    'Egypt': 0.02,
    'Nigeria': 0.02,
    'South Africa': 0.01,
    'Australia': 0.02,
    'Canada': 0.02,
    'South Korea': 0.01,
    'Italy': 0.01,
    'Spain': 0.01,
    'Saudi Arabia': 0.01,
    'Turkey': 0.01
}

def get_nationality_locale(nationality):
    """Get appropriate locale for the given nationality with fuzzy matching"""
    # First try exact match
    if nationality in NATIONALITY_NAME_MAPPING:
        return NATIONALITY_NAME_MAPPING[nationality]

    # Then try partial matching
    for country, locale in NATIONALITY_NAME_MAPPING.items():
        if country.lower() in nationality.lower():
            return locale

    # Finally try to find the most likely match
    for country in NATIONALITY_NAME_MAPPING:
        if any(part.lower() in nationality.lower()
               for part in country.split()):
            return NATIONALITY_NAME_MAPPING[country]

    return 'en_US'  # Default to US English

def generate_nationality():
    """Generate nationality based on weighted distribution"""
    countries = list(COUNTRY_WEIGHTS.keys())
    weights = list(COUNTRY_WEIGHTS.values())
    return random.choices(countries, weights=weights, k=1)[0]

def generate_gender_specific_name(gender, nationality):
    """Generate a name appropriate for the gender and nationality"""
    locale = get_nationality_locale(nationality)
    local_faker = Faker(locale)

    try:
        if gender.lower() == 'male':
            return local_faker.name_male()
        else:
            return local_faker.name_female()
    except:
        # Fallback to standard name generation if locale-specific fails
        if gender.lower() == 'male':
            return fake.name_male()
        else:
            return fake.name_female()

def generate_hash(text):
    """Generate MD5 hash for text uniqueness checking"""
    return hashlib.md5(text.encode()).hexdigest()

def ensure_unique_augmentation(original_text, cache, max_attempts=5):
    """Ensure augmented text is unique and meaningful"""
    attempts = 0
    while attempts < max_attempts:
        try:
            augmented = text_augmenter.augment(original_text)[0]
            text_hash = generate_hash(augmented)

            if text_hash not in cache and len(augmented.split()) >= len(original_text.split())/2:
                cache.add(text_hash)
                return augmented
        except:
            pass

        # Fallback: intelligent word swapping
        words = original_text.split()
        if len(words) > 1:
            idx1, idx2 = random.sample(range(len(words)), 2)
            words[idx1], words[idx2] = words[idx2], words[idx1]
            augmented = ' '.join(words)
            text_hash = generate_hash(augmented)
            if text_hash not in cache:
                cache.add(text_hash)
                return augmented

        attempts += 1

    # Final fallback - return original with marker
    return f"[AUGMENTED] {original_text}"

def generate_academic_level(age=None):
    """Generate realistic academic level based on age or randomly"""
    if age and age >= 22:
        # Higher probability for upper years if older
        levels = ['First', 'Second', 'Third', 'Fourth', 'Fifth']
        weights = [0.1, 0.15, 0.25, 0.3, 0.2]
    else:
        # Standard distribution
        levels = ['First', 'Second', 'Third', 'Fourth', 'Fifth']
        weights = [0.3, 0.25, 0.2, 0.15, 0.1]

    return random.choices(levels, weights=weights, k=1)[0]

def generate_realistic_gpa(year, student_id, all_gpas):
    """
    Generates realistic GPA that:
    - Progresses with academic level
    - Ensures uniqueness within tolerance
    - Adds diversity and avoids close duplicates
    """
    # Multiple GPA clusters for diversity
    year_params = {
        "First": [(2.3, 0.6), (2.6, 0.5)],
        "Second": [(2.6, 0.5), (2.9, 0.4)],
        "Third": [(2.9, 0.4), (3.1, 0.3)],
        "Fourth": [(3.1, 0.3), (3.3, 0.2)],
        "Fifth": [(3.2, 0.25), (3.4, 0.2)],
    }

    max_attempts = 10
    for _ in range(max_attempts):
        mean, std_dev = random.choice(year_params[year])
        gpa = np.random.normal(mean, std_dev)
        gpa += np.random.uniform(-0.02, 0.02)  # Small jitter for diversity
        gpa = np.clip(gpa, 1.5, 3.999)

        if not any(abs(gpa - existing) < 0.03 for existing in all_gpas.values()):
            all_gpas[student_id] = gpa
            return round(gpa, 4)

    # Fallback
    return round(np.clip(np.random.normal(mean, std_dev), 1.5, 3.999), 4)

def generate_quality_row(row):
    new_row = row.copy()

    # First determine gender (50/50 chance if not specified)
    gender = row.get('Gender', random.choice(['Male', 'Female']))

    # Generate nationality if not specified
    nationality = row.get('Nationality', generate_nationality())

    # Generate name based on gender and nationality
    new_row['Student Name'] = generate_gender_specific_name(gender, nationality)

    # Generate age if not specified
    age = row.get('Age', np.random.randint(18, 25))
    new_row['Age'] = age

    # Generate academic level first (since GPA depends on it)
    new_row['Year'] = generate_academic_level(age)

    # Ensure unique augmented responses (unchanged from original)
    new_row['Reports'] = ensure_unique_augmentation(row['Reports'], response_cache)
    new_row['Resolution'] = ensure_unique_augmentation(row['Resolution'], resolution_cache)

    # Generate GPA based on academic level
    new_row['GPA'] = generate_realistic_gpa(
        year=new_row['Year'],
        student_id=row['Student_ID'],
        all_gpas=global_gpas
    )

    # Set demographic information
    new_row['Gender'] = gender
    new_row['Nationality'] = nationality

    # Unique identifiers
    new_row['Ticket ID'] = f"TKT-{fake.unique.random_number(digits=6)}"

    return new_row

# Initialize global GPA tracker
global_gpas = {}

# Generate augmented data with progress tracking
augmented_data = []
unique_tickets = set(df['Ticket ID'].tolist())

for _ in tqdm(range(10000 - len(df)), desc="Generating synthetic data"):
    while True:
        genre = random.choice(df['Genre'].unique())
        row = df[df['Genre'] == genre].sample(1).iloc[0]
        new_row = generate_quality_row(row)

        # Ensure ticket ID uniqueness
        if new_row['Ticket ID'] not in unique_tickets:
            unique_tickets.add(new_row['Ticket ID'])
            augmented_data.append(new_row)
            break

# Create final dataset
augmented_df = pd.concat([
    df,
    pd.DataFrame(augmented_data)
], ignore_index=True)

# Enhanced quality checks
print(f"\nData Generation Complete:")
print(f"Total records: {len(augmented_df)}")
print(f"Unique responses: {augmented_df['Reports'].nunique()} ({augmented_df['Reports'].nunique()/len(augmented_df):.1%})")
print(f"Unique resolutions: {augmented_df['Resolution'].nunique()} ({augmented_df['Resolution'].nunique()/len(augmented_df):.1%})")

print("\nNationality Distribution:")
print(augmented_df['Nationality'].value_counts(normalize=True).head(10))

print("\nName examples by nationality:")
sample_countries = random.sample(list(NATIONALITY_NAME_MAPPING.keys()), 5)
for country in sample_countries:
    male_name = generate_gender_specific_name('Male', country)
    female_name = generate_gender_specific_name('Female', country)
    print(f"{country}: Male - {male_name}, Female - {female_name}")

# Save with validation
augmented_df.to_csv('enhanced_student_complaints.csv', index=False)

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
Generating synthetic data: 100%|██████████| 9046/9046 [18:32<00:00,  8.13it/s]



Data Generation Complete:
Total records: 10000
Unique responses: 9994 (99.9%)
Unique resolutions: 9878 (98.8%)

Nationality Distribution:
Nationality
Egypt           0.2403
Mexico          0.2019
France          0.1685
Indonesia       0.1536
Soudan          0.1211
Yemen           0.0762
South Africa    0.0384
Name: proportion, dtype: float64

Name examples by nationality:
Australia: Male - Joshua Blackwell, Female - Karen Richards
Spain: Male - Graciano Gras, Female - Reina Gámez Niño
United States: Male - John Dunn, Female - Dana Rivera
Turkey: Male - David Fields, Female - Sharon Nelson
India: Male - Leroy Malone, Female - Taylor Hudson


In [ ]:
import pandas as pd
import numpy as np
import random
from faker import Faker
from textattack.augmentation import WordNetAugmenter
from tqdm import tqdm
import hashlib
from faker.providers import person
import logging
from collections import defaultdict

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Set random seeds for reproducibility
random.seed(42)
np.random.seed(42)

# Initialize Faker with enhanced configurations
fake = Faker()
text_augmenter = WordNetAugmenter()

# Add nationality-specific name providers
fake.add_provider(person)

class DataGenerator:
    def __init__(self):
        self.response_cache = set()
        self.resolution_cache = set()
        self.ticket_cache = set()
        self.gpa_tracker = defaultdict(dict)

        # Enhanced nationality to name mapping with proper locales
        self.NATIONALITY_NAME_MAPPING = {
            'United States': 'en_US',
            'United Kingdom': 'en_GB',
            'China': 'en_GB',
            'India': 'en_GB',
            'Japan': 'en_GB',
            'Germany': 'de_DE',
            'France': 'en_GB',
            'Brazil': 'pt_BR',
            'Mexico': 'es_MX',
            'Russia': 'ru_RU',
            'Egypt': 'ar_EG',
            'Nigeria': 'en_GB',
            'South Africa': 'en_GB',
            'Australia': 'en_AU',
            'Canada': 'en_CA',
            'South Korea': 'en_GB',
            'Italy': 'it_IT',
            'Spain': 'es_ES',
            'Saudi Arabia': 'ar_SA',
            'Turkey': 'en_GB'
        }

        # Country weights for more realistic distribution
        self.COUNTRY_WEIGHTS = {
            'United States': 0.25,
            'United Kingdom': 0.1,
            'China': 0.15,
            'India': 0.15,
            'Japan': 0.05,
            'Germany': 0.05,
            'France': 0.05,
            'Brazil': 0.03,
            'Mexico': 0.03,
            'Russia': 0.02,
            'Egypt': 0.02,
            'Nigeria': 0.02,
            'South Africa': 0.01,
            'Australia': 0.02,
            'Canada': 0.02,
            'South Korea': 0.01,
            'Italy': 0.01,
            'Spain': 0.01,
            'Saudi Arabia': 0.01,
            'Turkey': 0.01
        }

    def get_nationality_locale(self, nationality):
        """Get appropriate locale for the given nationality with exact matching"""
        return self.NATIONALITY_NAME_MAPPING.get(nationality, 'en_US')

    def generate_nationality(self):
        """Generate nationality based on weighted distribution"""
        countries = list(self.COUNTRY_WEIGHTS.keys())
        weights = list(self.COUNTRY_WEIGHTS.values())
        return random.choices(countries, weights=weights, k=1)[0]

    def generate_gender_specific_name(self, gender, nationality):
        """Generate a name appropriate for the gender and nationality"""
        locale = self.get_nationality_locale(nationality)

        try:
            local_faker = Faker(locale)
            if gender.lower() == 'male':
                return local_faker.name_male()
            else:
                return local_faker.name_female()
        except Exception as e:
            logger.warning(f"Name generation failed for {nationality} ({locale}): {e}")
            # Fallback to standard name generation
            if gender.lower() == 'male':
                return fake.name_male()
            else:
                return fake.name_female()

    @staticmethod
    def generate_hash(text):
        """Generate MD5 hash for text uniqueness checking"""
        return hashlib.md5(text.encode()).hexdigest()

    def ensure_unique_augmentation(self, original_text, cache, max_attempts=5):
        """Ensure augmented text is unique and meaningful"""
        attempts = 0
        while attempts < max_attempts:
            try:
                augmented = text_augmenter.augment(original_text)[0]
                text_hash = self.generate_hash(augmented)

                if (text_hash not in cache and
                    len(augmented.split()) >= len(original_text.split())/2 and
                    augmented.lower() != original_text.lower()):
                    cache.add(text_hash)
                    return augmented
            except Exception as e:
                logger.debug(f"Augmentation failed: {e}")

            # Fallback: intelligent word swapping
            words = original_text.split()
            if len(words) > 1:
                idx1, idx2 = random.sample(range(len(words)), 2)
                words[idx1], words[idx2] = words[idx2], words[idx1]
                augmented = ' '.join(words)
                text_hash = self.generate_hash(augmented)
                if text_hash not in cache:
                    cache.add(text_hash)
                    return augmented

            attempts += 1

        # Final fallback - return original with marker
        return f"[AUGMENTATION_FAILED] {original_text}"

    def generate_academic_level(self, age=None):
        """Generate realistic academic level based on age or randomly"""
        if age and age >= 22:
            levels = ['First', 'Second', 'Third', 'Fourth', 'Fifth']
            weights = [0.1, 0.15, 0.25, 0.3, 0.2]
        else:
            levels = ['First', 'Second', 'Third', 'Fourth', 'Fifth']
            weights = [0.3, 0.25, 0.2, 0.15, 0.1]

        return random.choices(levels, weights=weights, k=1)[0]

    def generate_realistic_gpa(self, year, student_id):
        """
        Generates realistic GPA that:
        - Progresses with academic level
        - Ensures uniqueness within tolerance
        """
        # Clear GPA tracker if it gets too large
        if len(self.gpa_tracker) > 10000:
            self.gpa_tracker.clear()

        year_params = {
            "First": [(2.3, 0.6), (2.6, 0.5)],
            "Second": [(2.6, 0.5), (2.9, 0.4)],
            "Third": [(2.9, 0.4), (3.1, 0.3)],
            "Fourth": [(3.1, 0.3), (3.3, 0.2)],
            "Fifth": [(3.2, 0.25), (3.4, 0.2)],
        }

        max_attempts = 10
        for _ in range(max_attempts):
            mean, std_dev = random.choice(year_params[year])
            gpa = np.random.normal(mean, std_dev)
            gpa += np.random.uniform(-0.02, 0.02)  # Small jitter
            gpa = np.clip(gpa, 0.0, 4.0)  # Strict 0-4.0 range

            # Check against existing GPAs for this student
            if student_id not in self.gpa_tracker or not any(
                abs(gpa - existing) < 0.03
                for existing in self.gpa_tracker[student_id].values()
            ):
                if student_id not in self.gpa_tracker:
                    self.gpa_tracker[student_id] = {}
                self.gpa_tracker[student_id][year] = gpa
                return round(gpa, 2)  # Round to 2 decimal places

        # Fallback
        return round(np.clip(np.random.normal(mean, std_dev), 0.0, 4.0), 2)

    def validate_input_row(self, row):
        """Validate input row has required columns"""
        required_columns = ['Student_ID', 'Reports', 'Resolution', 'Genre']
        missing = [col for col in required_columns if col not in row]
        if missing:
            raise ValueError(f"Input row missing required columns: {missing}")

    def generate_quality_row(self, row):
        """Generate a high-quality synthetic row"""
        self.validate_input_row(row)

        new_row = row.copy()

        # Gender (50/50 chance if not specified)
        gender = row.get('Gender', random.choice(['Male', 'Female']))

        # Nationality (must be in our mapping)
        nationality = row.get('Nationality', self.generate_nationality())
        if nationality not in self.NATIONALITY_NAME_MAPPING:
            nationality = self.generate_nationality()

        # Name generation
        new_row['Student Name'] = self.generate_gender_specific_name(gender, nationality)

        # Age generation
        age = row.get('Age', np.random.randint(18, 25))
        new_row['Age'] = age

        # Academic level
        new_row['Year'] = self.generate_academic_level(age)

        # Unique augmented responses
        new_row['Reports'] = self.ensure_unique_augmentation(
            row['Reports'], self.response_cache)
        new_row['Resolution'] = self.ensure_unique_augmentation(
            row['Resolution'], self.resolution_cache)

        # GPA generation
        new_row['GPA'] = self.generate_realistic_gpa(
            year=new_row['Year'],
            student_id=row['Student_ID']
        )

        # Demographic information
        new_row['Gender'] = gender
        new_row['Nationality'] = nationality

        # Unique ticket ID
        while True:
            ticket_id = f"TKT-{fake.unique.random_number(digits=6)}"
            if ticket_id not in self.ticket_cache:
                self.ticket_cache.add(ticket_id)
                new_row['Ticket ID'] = ticket_id
                break

        return new_row

    def generate_synthetic_data(self, df, target_rows=10000):
        """Generate synthetic data until reaching target_rows"""
        if len(df) >= target_rows:
            logger.warning(f"Input already has {len(df)} rows (target: {target_rows})")
            return df.copy()

        augmented_data = []
        progress_bar = tqdm(total=target_rows - len(df), desc="Generating synthetic data")

        while len(df) + len(augmented_data) < target_rows:
            genre = random.choice(df['Genre'].unique())
            sample_row = df[df['Genre'] == genre].sample(1).iloc[0]

            try:
                new_row = self.generate_quality_row(sample_row)
                augmented_data.append(new_row)
                progress_bar.update(1)
            except Exception as e:
                logger.error(f"Error generating row: {e}")
                continue

        progress_bar.close()

        # Combine original and synthetic data
        result_df = pd.concat([df, pd.DataFrame(augmented_data)], ignore_index=True)

        # Data quality checks
        self.run_quality_checks(result_df)

        return result_df

    def run_quality_checks(self, df):
        """Run comprehensive data quality checks"""
        logger.info("\nData Generation Complete:")
        logger.info(f"Total records: {len(df)}")

        # Check uniqueness
        report_unique = df['Reports'].nunique()/len(df)
        resolution_unique = df['Resolution'].nunique()/len(df)
        logger.info(f"Unique responses: {df['Reports'].nunique()} ({report_unique:.1%})")
        logger.info(f"Unique resolutions: {df['Resolution'].nunique()} ({resolution_unique:.1%})")

        # Check nationality distribution
        logger.info("\nNationality Distribution:")
        logger.info(df['Nationality'].value_counts(normalize=True).head(10))

        # Validate all nationalities are in our mapping
        invalid_nationalities = set(df['Nationality']) - set(self.NATIONALITY_NAME_MAPPING.keys())
        if invalid_nationalities:
            logger.warning(f"Found nationalities not in mapping: {invalid_nationalities}")

        # Validate GPA range
        if df['GPA'].min() < 0 or df['GPA'].max() > 4.0:
            logger.error("Invalid GPA values detected!")

# Example Usage
if __name__ == "__main__":
    # Sample input dataframe (replace with your actual data)
    sample_data=pd.read_csv("/content/full_student_complaints.csv")
    input_df = pd.DataFrame(sample_data)

    # Generate synthetic data
    generator = DataGenerator()
    enhanced_df = generator.generate_synthetic_data(input_df, target_rows=10000)

    # Save output
    enhanced_df.to_csv('NEW_enhanced_student_complaints.csv', index=False)
    logger.info("Data successfully saved to enhanced_student_complaints.csv")

Streaming output truncated to the last 5000 lines.
Generating synthetic data:  51%|█████     | 4576/9046 [09:31<07:58,  9.34it/s]

Generating synthetic data:  51%|█████     | 4578/9046 [09:31<07:19, 10.17it/s]

Generating synthetic data:  51%|█████     | 4580/9046 [09:32<10:20,  7.20it/s]

Generating synthetic data:  51%|█████     | 4582/9046 [09:32<09:18,  7.99it/s]

Generating synthetic data:  51%|█████     | 4584/9046 [09:32<08:14,  9.03it/s]

Generating synthetic data:  51%|█████     | 4586/9046 [09:32<08:00,  9.28it/s]

Generating synthetic data:  51%|█████     | 4588/9046 [09:33<10:00,  7.42it/s]

Generating synthetic data:  51%|█████     | 4590/9046 [09:33<10:13,  7.26it/s]

Generating synthetic data:  51%|█████     | 4591/9046 [09:34<17:21,  4.28it/s]

Generating synthetic data:  51%|█████     | 4592/9046 [09:34<16:04,  4.62it/s]

Generating synthetic data:  51%|█████     | 4594/9046 [09:34<11:47,  6.29it/s]

Generating synthetic data:  51%|█████     | 4595/9046 [09:34<11:11,  

In [ ]:
augmented_df["Genre"].nunique()


11

In [ ]:
augmented_df.columns

Index(['Genre', 'Reports', 'Age', 'Year', 'Count', 'Gender', 'Nationality',
       'Resolution', 'Student Name', 'Ticket ID', 'Student_ID', 'GPA'],
      dtype='object')

In [ ]:
augmented_df.tail(50)

,Genre,Reports,Age,Year,Count,Gender,Nationality,Resolution,Student Name,Ticket ID,Student_ID,GPA
9950,Housing and Transportation,The lack of affordable housing options in this...,27,Third,1,Female,Yemen,Develop more affordable housing options on or ...,Nicole Lawrence,TKT-638904,STU1202,3.2645
9951,Student Affairs,I've had multiple outcome with the student aff...,27,Third,1,Male,Mexico,Increase staffing levels and improve training ...,Mauro Arellano Cortés,TKT-444978,STU1133,3.1749
9952,Food and Cantines,The food available on campus is too insistent....,20,Second,1,Female,Indonesia,Introduce rotating menu options in the campus ...,Meredith Lee,TKT-814614,STU1256,3.5553
9953,Activities and Travelling,I really enjoy exploring new vicinity and tryi...,20,Third,1,Male,Indonesia,constitute a foodie club to help students expl...,Mark Dean,TKT-487314,STU1231,3.4124
9954,Activities and Travelling,"I'm a transfer student, and I feel like I'm mi...",16,Second,1,Male,Yemen,Organize more trips and activities specificall...,Antonio Thomas,TKT-614332,STU1209,3.0310
9955,Health and Well-being Support,I've been live some medical symptoms that I'm ...,27,Fourth,1,Female,France,Provide spare or low-cost medical consultation...,Sarah Diaz,TKT-624912,STU1872,3.1830
9956,Career opportunities,The lack of job and internship opportunities a...,21,Fifth,1,Female,Egypt,provide mock interview sessions with HR profes...,Laura Rodriguez,TKT-8558,STU1663,2.4797
9957,Athletics and sports,The athletic facilities often not well-maintai...,23,Third,1,Female,Soudan,Create an athlete advisory council to voice st...,Donna Fleming,TKT-645918,STU1587,3.2542
9958,Student Affairs,The student affairs part seems to prioritize a...,21,Second,1,Male,Mexico,Shift the focus quality the student affairs of...,Raúl Nava de la Torre,TKT-442016,STU1118,2.2141
9959,Financial Support,I'm constantly apprehensive about having enou...,27,Third,1,Female,Egypt,Provide negotiation training workshops to help...,Christina Armstrong,TKT-218815,STU1720,3.4272
